In [137]:
%pip install matplotlib numpy ipywidgets ipympl

Note: you may need to restart the kernel to use updated packages.


In [138]:
%matplotlib ipympl
from ipywidgets import FloatSlider, interactive, IntSlider, Layout, widgets
import matplotlib.pyplot as plt
import numpy as N

RuntimeError: 'ipympl is not a recognised GUI loop or backend name

<h1>Dimensionamiento Sistemas de Baja Temperatura</h1>

<h2>Generalidades</h2>

La producción de ACS la aplicación práctica de la energía solar que mejor se adapta a las características de la misma.
Los niveles de temperaturas que son necesarios lograr (entre 40ºC y 50ºC) coinciden con los más apropiados para una buena eficacia del colector.
Es una necesidad que debe ser satisfecha durante los doce meses del año la inversión en el sistema solar se rentabilizará más rápidamente que aplicaciones estacionales:
- Calefacción en invierno
- Calentamiento de piscinas en verano.

Caracteristicas tipicas de instalaciones:
- Orientación paneles: Sur
- Inclinación: ~10° más que la latitud
- Menor inclinación favorece captación en verano
- Mayor inclinación favorece captación en invierno
- Evitar sombras
- Proteger del viento

<h2>Diseño del sistema</h2>

Variables importantes:
- Fracción Solar (SF): energía suministrada por el sistema solar sobre la energía total suministrada.
    \begin{equation}
    SF=\frac{Q_\mathrm{s}}{Q_\mathrm{s}+Q_\mathrm{tot}}
    \end{equation}
    - ${Q_\mathrm{tot}}$ demanda energetica total (kWh/año)
    <br>
    <br>
    \begin{equation}
    Q_\mathrm{tot} = Q_\mathrm{s}+Q_\mathrm{aux} 
    \end{equation}
    - ${Q_\mathrm{aux}}$ energía del sistema auxiliar (kWh/año)
    <br>
    <br>

- Eficiencia del Sistema (SE): energía suministrada por el sistema solar respecto a la radiación solar recibida en la superficie de captación en un año.
    \begin{equation}
    SF=\frac{Q_\mathrm{s}}{E_\mathrm{G}A}
    \end{equation}

    - $E_\mathrm{G}$ radiación total anual (kWh/m$2$.año)
    - A el area de la installacion (m$^2$)

El dimensionado del sistema solar depende de:
- Las condiciones climáticas locales (radiación y temperatura)
- La fracción solar deseada
- El rendimiento de la instalación

Objetivos típicos: La energía solar no cubra mas que la demanda en los periodos más bajos (Ejemplo: agua caliente sanitaria en verano)

Sistemas con excesivos dimensionados tiene los siguientes problemas:
- Largos periodos de estancamiento
- Mayor coste
- Rendimiento malo 

<h2>Metodo de diseño</h2>

1. Selección del punto de diseño (ubicación geográfica, inclinación del captador, día y hora, radiación y temperatura ambiente)

2. Cálculo de la demanda de agua caliente

3. Selección de la temperatura de entrada y salida del sistema solar y del fluido de trabajo

4. Selección del captador

5. Calculo del número de captadores para el día de diseño

6. Calculo del volumen de almacenamiento
 
7. Calculo de la energía auxiliar

<h3>1. Selección del punto de diseño</h3>

- Ubicación geográfica: Almería 36.8340° N, 2.4637° W 
- 21 de Julio mediodía solar: 960 W/m2 y 25ºC
- Inclinación: 10º mas de la latitud: 47º

<p align="center">
    <img src="Images_baja\Almeria_Summer_solstice.png" width='500'>
</p>




<h3>2. Cálculo de la demanda</h3>

Suponemos demanda de agua caliente para un edificio con 6 pisos con 5 viviendas cada una con 4 persona y consumo medio de agua caliente (50 l/persona.día a 45ºC), más el lavavajillas y la lavadora (dos veces por semana a 50ºC), agua fría a 10°C. La temperatura del almacenamiento es 60ºC.

Calor especifico del agua a 1 atm 4.2 Kj/KgK

Densidad del agua liquido a 1 atm: 1 Kg/m$^3$

In [ ]:
# Codigo calculo de demanda

style = {'description_width': 'initial'}
layout = Layout(width='100%')

pisos = IntSlider(min=1, max=10, value=6, step=1, description='Numero de pisos:', style=style, layout=layout)
viviendas = IntSlider(min=3, max=10, value=5, step=1, description='Numero de viviendas:', style=style, layout=layout)
personas = IntSlider(min=1, max=8, value=4, step=1, description='Numero de personas:', style=style, layout=layout)
consumo_persona = FloatSlider(min=30, max=100, value=50, step=0.2, description='Consumo per persona (l):', style=style, layout=layout)
temperatura_media = FloatSlider(min=40, max=50, value=45, step=0.1, description='Temperatura media de consumo (ºC):', style=style, layout=layout)
lavavajillas_semana = IntSlider(min=1, max=4, value=2, step=1, description='Usos de lavavajillas por semana:', style=style, layout=layout)
lavadora_semana = IntSlider(min=1, max=4, value=2, step=1, description='Usos de lavadora por semana:', style=style, layout=layout)
temperatura_aparatos = IntSlider(min=30, max=60, step=10, value=50, description='Temperatura de uso de aparatos (ºC):', style=style, layout=layout)
temperatura_fria = FloatSlider(min=5, max=20, value=10, step=0.1, description='Temperatura fria de la instalacion (ºC):', style=style, layout=layout)
temperatura_almacenamiento = FloatSlider(min=50, max=80, value=60, step=0.1, description='Temperatura del almacenamiento (ºC):', style=style, layout=layout)

Cv = 1e3*4.2 # capacidad termica volumetrica en kJ/m3
kWh = 1./3.6e3 # KJ

def demanda(p, v, n_p, c_p, T_m, lj_s, ld_s, T_ap, T_f, T_a):
    q_lj = 20./1000.*Cv*(T_ap-T_f) # energia por lavado lavavajillas en KJ
    q_ld = 30./1000.*Cv*(T_ap-T_f) # energia por lavado lavadora en KJ
    q_ap_d = (ld_s*q_ld+lj_s*q_lj)/7. # energia aparatos diaria en KJ/d
    
    q_p_d = c_p/1000.*Cv*(T_m-T_f) # energia por persona diaria en KJ/d
    q_tot_v = (q_ap_d + n_p*q_p_d) # energia total de una vivienda diaria en kWh/d
    v_tot_v = q_tot_v/(Cv*(T_a-T_f))*1e3 # volumen de almacenamioento por una vivienda a la temperatura de almacenamiento (l/d)
    
    q_ed = p*v*q_tot_v #energia total del piso diaria en KJ/d
    v_ed = q_ed/(Cv*(T_a-T_f))*1e3 # volumen de almacenamioento para el edificio a la temperatura de almacenamiento (l/d)
    
    print('Energía total diaria para una vivienda: %.3f kWh/d\nVolumen de agua total diario para una vivienda: %.1f l\nEnergía total diaria para el edificio: %.1f kWh/d\nVolumen de agua total diario para el edificio: %.1f l'%(q_tot_v*kWh, v_tot_v, q_ed*kWh, v_ed))
    
    output1 = widgets.Output()
    with output1:
        fig1, ax = plt.subplots()
        ax.bar([1,2], [q_tot_v*kWh, q_ed*kWh])
        plt.ylabel('Consumo kWh/d')
       
    output2 = widgets.Output()
    with output2:
        fig2, ax = plt.subplots()
        ax.bar([1,2], [v_tot_v, v_ed])
        plt.ylabel('Consumo l/d')

    return 

sliders = interactive(demanda, p=pisos, v=viviendas, n_p=personas, c_p=consumo_persona, T_m=temperatura_media, lj_s=lavavajillas_semana, ld_s=lavadora_semana, T_ap=temperatura_aparatos, T_f=temperatura_fria, T_a=temperatura_almacenamiento)

layout_figs = Layout(height='25%')
    
labels = ['Q_v', 'V_v', 'Q_e', 'V_e']
widgets.HBox([sliders, VBox([output1, output2])])

- T in= 10ºC (temperatura fría)
- T out= 60ºC (temperatura requerida en almacenamiento)
- Fluido= Agua (+glicol)

<h3>4. Selección del captador</h3>

- Modelo: FP Vitosol 100
- Área de apertura: 2.33 m
- Ancho: 1.056 m
- Costo: 1000 €/m$^2$
- Caudal del captador: 0.02 kg/s.m$^2$
- Eficiencia: 
\begin{equation}
\eta=0.84-3.36\frac{T_\mathrm{m}-T_\mathrm{amb}}{G}-0.013\frac{(T_\mathrm{m}-T_\mathrm{amb})^2}{G}
\end{equation}

<p align="center">
    <img src="Images_baja\colector.png" width='265'>
    <img src="Images_baja\Colector_corte.png" width='400'>
</p>


<h3>5. Calculo del número de captadores para el día de diseño</h3>

<h3>6. Calculo del volumen de almacenamiento</h3>

<h3>7. Calculo de la energía auxiliar</h3>

<h2>Distancia entre captadores</h2>

Se calcula en el mediodia solar para el día del año con menor altura solar: 21 de diciembre (hemisferio norte)/ 21 de julio (hemisferio sur)
<p align="center">
    <img src="Images_baja\collector_angles.png">
</p>